In [1]:
import numpy as np
import nibabel as nib
from nibabel.processing import resample_from_to

In [2]:
affine_target = np.array([
    [-2,  0,  0,   96], 
    [ 0,  2,  0, -112],
    [ 0,  0,  2,  -90],
    [ 0,  0,  0,    1]
])

##### prepare fmri data

In [ ]:
fmri_img = nib.load('./MNI152_fmri_data.nii.gz') # TBD

fmri_data = fmri_img.get_fdata(); affine = fmri_img.affine; header = fmri_img.header.copy()
fmri_cropped = fmri_data[..., :300]; header.set_data_shape(fmri_cropped.shape); cropped_img = nib.Nifti1Image(fmri_cropped, affine, header)
resampled_fmri = resample_from_to(cropped_img, ((96, 96, 96, 300), affine_target), order = 3) # if fmri data has 2mm isotropic spatial resolution, use order = 0 for quicker resampling

nib.save(resampled_fmri, './prepared_rfmri_data.nii.gz') # TBD

##### prepare tissue mask

In [ ]:
aseg_img = nib.load('./MNI152_aparc+aseg.nii.gz') # TBD
aseg_data = aseg_img.get_fdata()

res = np.zeros_like(aseg_data)
res += 1 * ((aseg_data >= 1000) & (aseg_data <= 1035)).astype(int); res += 11 * ((aseg_data >= 2000) & (aseg_data <= 2035)).astype(int)
res += 2 * (aseg_data == 2).astype(int); res += 12 * (aseg_data == 41).astype(int)
for i in [10, 11, 12, 13, 17, 18, 26, 27, 28, 75, 96]: res += 3 * (aseg_data == i).astype(int)
for i in [49, 50, 51, 52, 53, 54, 58, 59, 60, 76, 97]: res += 13 * (aseg_data == i).astype(int)
res += 20 * ((aseg_data >= 251) & (aseg_data <= 255)).astype(int)

new_img = nib.Nifti1Image(res, affine = aseg_img.affine)
resampled_img = resample_from_to(new_img, ((96, 96, 96), affine_target), order = 0)

nib.save(resampled_img, './prepared_tissue_mask.nii.gz') # TBD